# Speeding the Code Up to 300 times Faster Compared with Original Code

In [2]:
import os
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

## 1. Original Code
- All the descriptions are removed for the clear comparision
- The source of the functions are ASM.py file
- Due to the difference in unit (ft/sec) and axis setting (x-position and time stamp starts from 0, proceeds to positive value), the hyper parameters are adjusted

- Original paper that ASM.py is based on: An adaptive smoothing method for traffic state identification from incomplete information [link to paper](https://link.springer.com/chapter/10.1007/978-3-662-07969-0_33)

- smooth_x_window $\sigma$
    - Paper → 0.60 km = 0.373 mi = 1,968.5 ft
    - Example code → 0.24 km = 0.150 mi = 792.0 ft
    - 1,000 ft is used
- smooth_t_window $\tau$
    - Paper → 1.1 min = 66 sec
    - Example code → 0.6 min = 36 sec
    - 60 sec is used
- c_free
    - Paper → +80 km/h = +49.7 mi/h = +72.9 ft/s
    - Example code → -69.2 km/h = -43.0 mi/h = -63.1 ft/s
    - Based on the data analysis, 90 ft/s is used
- c_cong
    - Paper → -15.0 km/h = -9.3 mile/h = -13.7 ft/s
    - Example code → +20.9 km/h = +13 mile/h = +19.1 ft/s
    - Based on the data analysis, -15 ft/s is used
- V_c
    - Paper → 60 km/h → 37.3 mile/h → 54.7 ft/s
    - Example code → 60 km/h → approximated to 36.0 mile/h → 52.8 ft/s
    - 55 ft/s  is used
- $\Delta V$ (Transition region)
    - Paper → 20 km/h → 12.43 mile/h → 18.2269 ft/s
    - Example code → 20 km/h → 12.43 mile/h → 18.2269 ft/s
    - 18.227 ft/s  is used

In [6]:
def beta_free_flow(x, t, x_s, t_s, x_win, t_win, c_free=90):

    dt = t-t_s- (x-x_s)/c_free
    dx = x-x_s
    return np.exp(-(np.abs(dx)/x_win + np.abs(dt)/t_win))


def beta_cong_flow(x, t, x_s, t_s, x_win, t_win, c_cong=-15):

    dt = t-t_s- (x-x_s)/c_cong
    dx = x-x_s
    return np.exp(-(np.abs(dx)/x_win + np.abs(dt)/t_win))


def EGTF(x, t, smooth_x_window, smooth_t_window, speed_raw_df):

    speed = speed_raw_df[(np.abs(speed_raw_df.t - t)<=(smooth_t_window/2)) & (np.abs(speed_raw_df.x - x)<=(smooth_x_window/2))]
        
    speed = speed.copy()
    EGTF_v_free = 80
    EGTF_v_cong = 80

    speed['beta_free'] = speed.apply(
        lambda v: beta_free_flow(x, t, v.x, v.t, smooth_x_window, smooth_t_window),
        axis=1)
    speed['beta_cong'] = speed.apply(
        lambda v: beta_cong_flow(x, t, v.x, v.t, smooth_x_window, smooth_t_window),
        axis=1)
    if((sum(speed.beta_free)!=0) & (sum(speed.beta_cong)!=0)):
        EGTF_v_free = sum(speed.beta_free * speed.speed) / sum(speed.beta_free)
        EGTF_v_cong = sum(speed.beta_cong * speed.speed) / sum(speed.beta_cong)
    v = min(EGTF_v_free,EGTF_v_cong)
    tanh_term = np.tanh((55-v) / 18.227)
    w = 0.5*(1+tanh_term)
    return w*EGTF_v_cong + (1-w)*EGTF_v_free


def smooth_raw_data(speed_raw, dx, dt, smooth_x_window=1000, smooth_t_window=60):

    EGTF_speed = pd.DataFrame(speed_raw)
    EGTF_speed.columns=['t_index','x_index','speed']
    EGTF_speed['t'] = dt*EGTF_speed['t_index']
    EGTF_speed['x'] = dx*EGTF_speed['x_index']
    speed_raw_df  = EGTF_speed.dropna().copy()
    tqdm.pandas(desc="Processing EGTF")
    EGTF_speed['EGTF'] = EGTF_speed.progress_apply(lambda v: EGTF(v.x, v.t, smooth_x_window,smooth_t_window,speed_raw_df), axis=1)
    EGTF_speed.columns = ['t','x','raw_speed','time','milemarker','speed']
    return EGTF_speed

- The speed_mat file is a 2D array. 
- X-axis is a time-axis and Y-axis is a x-position.
- The value of each element is speed.
- For a quick and clear comparision, use the part of the original array (100 by 120)
- 100 by 120 is equal to 100 sec by 1200 ft

In [3]:
speed_mat = np.load('selected/lane_matrix/westbound_11_29_21_23_lane_1.npy')

# cut 100 by 120
speed_mat = speed_mat[:100, :120]

time_w, space_w = speed_mat.shape

t_col = np.arange(time_w).repeat(space_w)
x_col = np.tile(np.arange(space_w), time_w)

speed_df = pd.DataFrame({'t': t_col, 'x': x_col, 'speed': speed_mat.flatten()})
speed_df.to_csv('selected/lane_matrix/westbound_11_29_21_23_speed_raw_lane_1.csv', index=False)

In [ ]:
dx = 10 
dt = 1

t0 = time.time()
smooth_speed = smooth_raw_data(speed_df, dx, dt, 1000, 60)
print(f"Smoothing compute time: {time.time() - t0}s")

smooth_speed.to_csv('selected/lane_matrix/westbound_11_29_21_23_speed_raw_lane_1_smooth_1.csv', index=False)

- Total time consumption can vary, but usually around 1,000 sec

## 2. Updated Code

## Interpolation

In [3]:
import numpy as np
from scipy.interpolate import griddata

def interpolate_zeros(data):
    # Get the x and y coordinates of all points
    x, y = np.indices(data.shape)
    
    # Get the coordinates of non-zero points
    nonzero_coords = np.column_stack((x[data != 0], y[data != 0]))
    nonzero_values = data[data != 0]
    
    # Get the coordinates of zero points and interpolate
    zero_coords = np.column_stack((x[data == 0], y[data == 0]))
    
    interpolated_values = griddata(nonzero_coords, nonzero_values, zero_coords, method='linear')
    
    # Identify NaN values in the interpolation result and fill
    nan_indices = np.isnan(interpolated_values)
    interpolated_values_nearest = griddata(nonzero_coords, nonzero_values, zero_coords, method='nearest')
    
    # Fill NaN values with nearest neighbor interpolation results
    interpolated_values[nan_indices] = interpolated_values_nearest[nan_indices]
    data[data == 0] = interpolated_values
    
    return data

lane_matrix_files = os.listdir('selected/lane_matrix')

for file in lane_matrix_files[56:]:

    speed_raw = np.load(f'selected/lane_matrix/{file}')

    t0 = time.time()
    speed_raw = interpolate_zeros(speed_raw)
    print(f"Interpolation compute time: {time.time() - t0}s for {file}")

    # save the smoothed speed matrix
    np.save(f'selected/lane_matrix_interpolated/{file}', speed_raw)

Interpolation compute time: 106.71404886245728s for westbound_11_22_21_23_lane_1.npy
Interpolation compute time: 87.07699036598206s for westbound_11_22_21_23_lane_2.npy
Interpolation compute time: 74.81298422813416s for westbound_11_22_21_23_lane_3.npy
Interpolation compute time: 80.49079275131226s for westbound_11_22_21_23_lane_4.npy
Interpolation compute time: 95.05388259887695s for westbound_11_22_23_25_lane_1.npy
Interpolation compute time: 90.2676293849945s for westbound_11_22_23_25_lane_2.npy
Interpolation compute time: 71.28343653678894s for westbound_11_22_23_25_lane_3.npy
Interpolation compute time: 65.74023056030273s for westbound_11_22_23_25_lane_4.npy
Interpolation compute time: 94.41172337532043s for westbound_11_23_21_23_lane_1.npy
Interpolation compute time: 87.73641967773438s for westbound_11_23_21_23_lane_2.npy
Interpolation compute time: 74.00520300865173s for westbound_11_23_21_23_lane_3.npy
Interpolation compute time: 68.88186407089233s for westbound_11_23_21_23_lan

In [5]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

processed_files = os.listdir("selected/lane_matrix_interpolated")

for processed_file in processed_files:
    if processed_file[-4:] != '.npy':
        continue
    
    direction = processed_file[:9]

    print(processed_file)

    speedmap = np.load(f'selected/lane_matrix_interpolated/{processed_file}')

    jet = matplotlib.colormaps['jet']
    reversed_jet_colors = jet(np.linspace(1, 0, jet.N))  # Reverse the colormap
    reversed_jet_colors[:1, -1] = 0  # Set the alpha of the first color to 0 (transparent for NaN values)
    inverted_jet_with_blank = ListedColormap(reversed_jet_colors)

    fig, ax = plt.subplots(figsize=(18, 10))
    # show the matrix plot
    im = ax.imshow(np.flip(speedmap.T, axis=0), aspect='auto', cmap=inverted_jet_with_blank, vmin=0, vmax=135)
    ax.set_title(f'{processed_file[:-4]}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (ft)')
    plt.colorbar(im, ax=ax)
    fig.savefig(f'selected/image/interpolated/{processed_file[:-4]}.png', dpi=300)
    plt.close(fig)
    plt.cla()
    plt.clf()

westbound_11_22_21_23_lane_1.npy
westbound_11_22_21_23_lane_2.npy
westbound_11_22_21_23_lane_3.npy
westbound_11_22_21_23_lane_4.npy
westbound_11_22_23_25_lane_1.npy
westbound_11_22_23_25_lane_2.npy
westbound_11_22_23_25_lane_3.npy
westbound_11_22_23_25_lane_4.npy
westbound_11_23_21_23_lane_1.npy
westbound_11_23_21_23_lane_2.npy
westbound_11_23_21_23_lane_3.npy
westbound_11_23_21_23_lane_4.npy
westbound_11_23_23_25_lane_1.npy
westbound_11_23_23_25_lane_2.npy
westbound_11_23_23_25_lane_3.npy
westbound_11_23_23_25_lane_4.npy
westbound_11_28_21_23_lane_1.npy
westbound_11_28_21_23_lane_2.npy
westbound_11_28_21_23_lane_3.npy
westbound_11_28_21_23_lane_4.npy
westbound_11_28_23_25_lane_1.npy
westbound_11_28_23_25_lane_2.npy
westbound_11_28_23_25_lane_3.npy
westbound_11_28_23_25_lane_4.npy
westbound_11_29_21_23_lane_1.npy
westbound_11_29_21_23_lane_2.npy
westbound_11_29_21_23_lane_3.npy
westbound_11_29_21_23_lane_4.npy
westbound_11_29_23_25_lane_1.npy
westbound_11_29_23_25_lane_2.npy
westbound_

<Figure size 640x480 with 0 Axes>

## Smoothing

In [7]:
import os
import numpy as np
import pandas as pd

lane_matrix_files = os.listdir('selected/lane_matrix_interpolated')

for file in lane_matrix_files:

    speed_raw = np.load(f'selected/lane_matrix_interpolated/{file}')

    t0 = time.time()

    time_w, space_w = speed_raw.shape

    x_scale = 10
    t_win = 4 # 60 sec
    x_win = 16 # 1000 ft
    c_free = 90
    c_cong = -15

    # create time matrix and space matrix
    time_mat = np.arange(time_w).repeat(space_w).reshape(time_w, -1)
    space_mat = np.asarray([np.arange(space_w)]*(time_w)) * x_scale

    speed_smooth = np.zeros((time_w, space_w))

    for time_idx in range(time_w):
        for space_idx in range(space_w):

            EGTF_v_free = 80
            EGTF_v_cong = 80

            # time_idx and real time is 1:1, space_idx and real space is 1:10
            time_val = time_idx
            space_val = space_idx * x_scale

            t_start = max(time_idx-(t_win//2), 0)
            t_end = min(time_idx+(t_win//2)+1, time_w)
            
            x_start = max(space_idx-(x_win//2), 0)
            x_end = min(space_idx+(x_win//2)+1, space_w)
            
            speed_slice = speed_raw[t_start:t_end, x_start:x_end]
            time_slice = time_mat[t_start:t_end, x_start:x_end]
            space_slice = space_mat[t_start:t_end, x_start:x_end]
            
            time_diff = time_val-time_slice
            dx_mat = space_val-space_slice

            dt_mat_free = time_diff - dx_mat/c_free
            dt_mat_cong = time_diff - dx_mat/c_cong

            beta_mat_free = np.exp(-(np.abs(dt_mat_free)/t_win + np.abs(dx_mat)/(x_win*x_scale)))
            beta_mat_cong = np.exp(-(np.abs(dt_mat_cong)/t_win + np.abs(dx_mat)/(x_win*x_scale)))
            
            if ((sum(sum(beta_mat_free)) != 0) & (sum(sum(beta_mat_cong)) != 0)):
                EGTF_v_free = sum(sum(beta_mat_free * speed_slice)) / sum(sum(beta_mat_free))
                EGTF_v_cong = sum(sum(beta_mat_cong * speed_slice)) / sum(sum(beta_mat_cong))
            
            v = min(EGTF_v_free, EGTF_v_cong)
            tanh_term = np.tanh((55 - v) / 18.227)
            w = 0.5*(1+tanh_term)
            speed_smooth[time_idx, space_idx] = w*EGTF_v_cong + (1-w)*EGTF_v_free

    print(f"Smoothing compute time: {time.time() - t0}s")

    # save the smoothed speed matrix
    np.save(f'selected/smooth/{file}', speed_smooth)
    print(f"Saved smoothed speed matrix to selected/smoothed/{file}")

Smoothing compute time: 1125.7935602664948s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_21_23_lane_1.npy
Smoothing compute time: 1121.7911179065704s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_21_23_lane_2.npy
Smoothing compute time: 1126.3717141151428s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_21_23_lane_3.npy
Smoothing compute time: 1119.7035830020905s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_21_23_lane_4.npy
Smoothing compute time: 1117.4645681381226s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_23_25_lane_1.npy
Smoothing compute time: 1113.9616100788116s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_23_25_lane_2.npy
Smoothing compute time: 1115.0656735897064s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_23_25_lane_3.npy
Smoothing compute time: 1113.2944853305817s
Saved smoothed speed matrix to selected/smoothed/westbound_11_22_23_25_lan

In [6]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

processed_files = os.listdir("selected/smooth")

for processed_file in processed_files:
    if processed_file[-4:] != '.npy':
        continue
    
    direction = processed_file[:9]

    print(processed_file)

    speedmap = np.load(f'selected/smooth/{processed_file}')

    jet = matplotlib.colormaps['jet']
    reversed_jet_colors = jet(np.linspace(1, 0, jet.N))  # Reverse the colormap
    reversed_jet_colors[:1, -1] = 0  # Set the alpha of the first color to 0 (transparent for NaN values)
    inverted_jet_with_blank = ListedColormap(reversed_jet_colors)

    fig, ax = plt.subplots(figsize=(18, 10))
    # show the matrix plot
    im = ax.imshow(np.flip(speedmap.T, axis=0), aspect='auto', cmap=inverted_jet_with_blank, vmin=0, vmax=135)
    ax.set_title(f'{processed_file[:-4]}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (ft)')
    plt.colorbar(im, ax=ax)
    fig.savefig(f'selected/image/smooth/{processed_file[:-4]}.png', dpi=300)
    plt.close(fig)
    plt.cla()
    plt.clf()

westbound_11_22_21_23_lane_1.npy


<Figure size 640x480 with 0 Axes>

# Aggregation

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


lane_matrix_files = os.listdir('selected/smooth')

for processed_file in lane_matrix_files:
    if processed_file[-4:] != '.npy':
        continue
    
    lane_idx = processed_file[-5]
    day = processed_file[13:15]

    speed_smooth = np.load(f'selected/smooth/{processed_file}')
    time_w, space_w = speed_smooth.shape

    time_block, space_block = 5, 20

    # for a time axis, truncate from the beginning
    # for a space axis, truncate from the end
    time_w_res = time_w % time_block
    space_w_res = space_w % space_block

    speed_smooth = speed_smooth[time_w_res:, :-space_w_res]
    time_w, space_w = speed_smooth.shape

    # Define block size
    block_size = (time_block, space_block)

    # Reshape the speed_smooth into blocks
    speed_smooth_aggregate = speed_smooth.reshape(speed_smooth.shape[0] // block_size[0], block_size[0],
                                                  speed_smooth.shape[1] // block_size[1], block_size[1])
    
    # Mask zeros and compute the mean
    masked_speed_smooth = np.ma.masked_equal(speed_smooth_aggregate, 0)
    mean_values = masked_speed_smooth.mean(axis=(1, 3))

    # change masked to 0
    mean_values = np.ma.filled(mean_values, 0)

    jet = matplotlib.colormaps['jet']
    reversed_jet_colors = jet(np.linspace(1, 0, jet.N))  # Reverse the colormap
    reversed_jet_colors[:1, -1] = 0  # Set the alpha of the first color to 0 (transparent for NaN values)
    inverted_jet_with_blank = ListedColormap(reversed_jet_colors)

    fig, ax = plt.subplots(figsize=(18, 10))
    # show the matrix plot
    im = ax.imshow(np.flip(mean_values.T, axis=0), aspect='auto', cmap=inverted_jet_with_blank, vmin=0, vmax=135)
    ax.set_title(f'{processed_file[:-4]}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (ft)')
    plt.colorbar(im, ax=ax)
    fig.savefig(f'selected/image/aggregate/{processed_file[:-4]}.png', dpi=300)
    plt.close(fig)
    plt.cla()
    plt.clf()

    if lane_idx == '1':
        stack_mean_values = mean_values

    elif (lane_idx == '2'):
        stack_mean_values = np.stack((stack_mean_values, mean_values))

    elif lane_idx == '3':
        mean_values = np.expand_dims(mean_values, axis=0)
        stack_mean_values = np.concatenate((stack_mean_values, mean_values), axis=0)

    elif lane_idx == '4':
        mean_values = np.expand_dims(mean_values, axis=0)
        stack_mean_values = np.concatenate((stack_mean_values, mean_values), axis=0)
        print(stack_mean_values.shape)
        np.save(f'selected/aggregate/{processed_file[:21]}_aggregate.npy', stack_mean_values)
        print(f"Saved aggregated speed matrix to selected/aggregate/{processed_file[:21]}_aggregate.npy")

(4, 1440, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_22_21_23_aggregate.npy
(4, 1438, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_22_23_25_aggregate.npy
(4, 1440, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_23_21_23_aggregate.npy
(4, 1438, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_23_23_25_aggregate.npy
(4, 1440, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_28_21_23_aggregate.npy
(4, 1438, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_28_23_25_aggregate.npy
(4, 1440, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_29_21_23_aggregate.npy
(4, 1438, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_29_23_25_aggregate.npy
(4, 1439, 110)
Saved aggregated speed matrix to selected/aggregate/westbound_11_30_21_23_aggregate.npy
(4, 1438, 110)
Saved aggregated speed matrix to selected/aggregate/westbo

<Figure size 640x480 with 0 Axes>

# Create a Random Sample

In [5]:
import os
import numpy as np

num_samples_per_file = 715
sample_length = 64
num_of_lanes = 4

# Use the random number that we got from before interpolation
files_before_interpolation = os.listdir('selected/aggregate/before_interpolation')
files_after_interpolation = os.listdir('selected/aggregate')

i = 0
file_count = 0

for file_bi, file_ai in zip(files_before_interpolation, files_after_interpolation):
    if (file_bi[-4:] != '.npy') and (file_ai[-4:] != '.npy'):
        continue

    speed_bi = np.load(f'selected/aggregate/before_interpolation/{file_bi}')
    speed_ai = np.load(f'selected/aggregate/{file_ai}')
    lanes_bi, time_w_bi, space_w_bi = speed_bi.shape
    lanes_ai, time_w_ai, space_w_ai = speed_ai.shape
    if (lanes_bi != lanes_ai) or (time_w_bi != time_w_ai) or (space_w_bi != space_w_ai):
        print(f"Error: {file_bi} and {file_ai} have different shapes")
    
    print(lanes_bi, time_w_bi, space_w_bi)
    sample_time_w = time_w_bi - sample_length
    sample_space_w = space_w_bi - sample_length

    random_range = np.random.randint(0, sample_time_w*sample_space_w, num_samples_per_file)
    random_range_time = random_range // sample_space_w
    random_range_space = random_range % sample_space_w

    while (i<(num_samples_per_file*(file_count+1)) and i<10000):
        for time_idx, space_idx in zip(random_range_time, random_range_space):
            sample = speed_bi[:, time_idx:time_idx+sample_length, space_idx:space_idx+sample_length]
            if np.count_nonzero(sample) <= (0.9 * num_of_lanes * sample_length * sample_length):
                print("Skip")
                continue
            sample_save = speed_ai[:, time_idx:time_idx+sample_length, space_idx:space_idx+sample_length]
            np.save(f'selected/dataset/{i:04}.npy', sample_save)
            print(i)
            i += 1
            if i>=10000: break
        zero_count = (num_samples_per_file*(file_count+1)) - i
        random_range = np.random.randint(0, sample_time_w*sample_space_w, zero_count)
        random_range_time = random_range // sample_space_w
        random_range_space = random_range % sample_space_w
    
    file_count += 1

PermissionError: [Errno 13] Permission denied: 'selected/aggregate/before_interpolation'

In [ ]:
import os
import numpy as np

def ensure_readable(file_path):
    try:
        # Attempt to read the file
        with open(file_path, 'rb') as f:
            np.load(f)
    except PermissionError as e:
        print(f"PermissionError: {e}")
        # Change permissions to allow reading
        try:
            os.chmod(file_path, 0o644)
            print(f"Permissions changed for file: {file_path}")
        except Exception as e:
            print(f"Failed to change permissions: {e}")
            raise

num_samples_per_file = 715
sample_length = 64
num_of_lanes = 4

# Ensure directories are accessible
before_interpolation_dir = 'selected/aggregate/before_interpolation'
after_interpolation_dir = 'selected/aggregate'
output_dir = 'selected/dataset'

os.makedirs(output_dir, exist_ok=True)

files_before_interpolation = os.listdir(before_interpolation_dir)
files_after_interpolation = os.listdir(after_interpolation_dir)

files_after_interpolation = [file for file in files_after_interpolation if file[-4:] == '.npy']
files_before_interpolation = [file for file in files_before_interpolation if file[-4:] == '.npy']

# Ensure all files are readable
for file_bi in files_before_interpolation:
    ensure_readable(os.path.join(before_interpolation_dir, file_bi))
for file_ai in files_after_interpolation:
    ensure_readable(os.path.join(after_interpolation_dir, file_ai))

i = 0
file_count = 0

for file_bi, file_ai in zip(files_before_interpolation, files_after_interpolation):
    if not file_bi.endswith('.npy') or not file_ai.endswith('.npy'):
        continue

    speed_bi = np.load(os.path.join(before_interpolation_dir, file_bi))
    speed_ai = np.load(os.path.join(after_interpolation_dir, file_ai))
    lanes_bi, time_w_bi, space_w_bi = speed_bi.shape
    lanes_ai, time_w_ai, space_w_ai = speed_ai.shape
    if lanes_bi != lanes_ai or time_w_bi != time_w_ai or space_w_bi != space_w_ai:
        print(f"Error: {file_bi} and {file_ai} have different shapes")
        continue
    
    print(lanes_bi, time_w_bi, space_w_bi)
    sample_time_w = time_w_bi - sample_length
    sample_space_w = space_w_bi - sample_length

    random_range = np.random.randint(0, sample_time_w * sample_space_w, num_samples_per_file)
    random_range_time = random_range // sample_space_w
    random_range_space = random_range % sample_space_w

    while i < (num_samples_per_file * (file_count + 1)) and i < 10000:
        for time_idx, space_idx in zip(random_range_time, random_range_space):
            sample = speed_bi[:, time_idx:time_idx + sample_length, space_idx:space_idx + sample_length]
            if np.count_nonzero(sample) <= (0.95 * num_of_lanes * sample_length * sample_length):
                print("Skip")
                continue
            sample_save = speed_ai[:, time_idx:time_idx + sample_length, space_idx:space_idx + sample_length]
            np.save(os.path.join(output_dir, f'{i:04}.npy'), sample_save)
            print(i)
            i += 1
            if i >= 10000:
                break

        zero_count = (num_samples_per_file * (file_count + 1)) - i
        random_range = np.random.randint(0, sample_time_w * sample_space_w, zero_count)
        random_range_time = random_range // sample_space_w
        random_range_space = random_range % sample_space_w

    file_count += 1